In [1]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek
from langchain_core.runnables import RunnableBranch

RunnableBranch: 基于条件选择哪个分支继续运行 + 默认分支

In [2]:
llm = ChatDeepSeek(
    model="deepseek-chat", 
    temperature=0, 
    api_key=os.getenv("DEEPSEEK_API_KEY")
    )

## 设置不同菜系的处理链

In [3]:
sichuan_prompt = ChatPromptTemplate.from_template(
    "你是一位川菜大厨。请用正宗的川菜做法，回答关于 【{question}】的问题"
)

sichuan_chain = sichuan_prompt | llm | StrOutputParser()

cantonese_prompt = ChatPromptTemplate.from_template(
    "你是一位粤菜大厨。请用经典的粤菜做法，回答关于【{question}】的问题。"
)
cantonese_chain = cantonese_prompt | llm | StrOutputParser()

# 定义备用通用链
general_prompt = ChatPromptTemplate.from_template(
    "你是一个美食助手。请回答关于【{question}】的问题。"
)
general_chain = general_prompt | llm | StrOutputParser()


## 创建路由链

In [6]:
classifier_prompt = ChatPromptTemplate.from_template(
    """根据用户问题中所提到的菜品, 将其分类为其中一个: ['川菜', '粤菜', '其它类别']
    禁止解释理由, 只需返回类别。
    问题：{question}
    """
)

classifier_chain = classifier_prompt | llm | StrOutputParser()

# 定义路由分支
route_branch = RunnableBranch(
    (lambda x: "川菜" in x['topic'], sichuan_chain),
    (lambda x: "粤菜" in x['topic'], cantonese_chain),
    general_chain
)

# 组合成完整路由链
full_router_chain = \
    {"topic": classifier_chain, "question": lambda x: x['question']} | route_branch

print("完整的路由链创建成功。\n")

完整的路由链创建成功。



## 运行演示查询

In [ ]:
demo_questions = [
    {"question": "麻婆豆腐怎么做？"},      # 应该路由到川菜
    {"question": "白切鸡的正宗做法是什么？"}, # 应该路由到粤菜
    {"question": "番茄炒蛋需要放糖吗？"}      # 应该路由到其他
]

for i, item in enumerate(demo_questions, 1):
    question = item["question"]
    print(f"\n--- 问题 {i}: {question} ---")

    try:
        # 获取路由决策
        topic = classifier_chain.invoke({"question": question})
        print(f"路由决策-topic: {topic}")
        print(item)

    except Exception as e:
        print(f"执行错误: {e}")


--- 问题 1: 麻婆豆腐怎么做？ ---
路由决策-topic: 川菜
{'question': '麻婆豆腐怎么做？'}

--- 问题 2: 白切鸡的正宗做法是什么？ ---
路由决策-topic: 粤菜
{'question': '白切鸡的正宗做法是什么？'}

--- 问题 3: 番茄炒蛋需要放糖吗？ ---
路由决策-topic: 其它类别
{'question': '番茄炒蛋需要放糖吗？'}


In [9]:
for i, item in enumerate(demo_questions, 1):
    item["question"] += "回答少于100字"
    question = item["question"]
    print(f"\n--- 问题 {i}: {question} ---")

    try:
        # 直接执行完整链路
        answer = full_router_chain.invoke(item)
        print(f"回答: {answer}")
        print(item)

    except Exception as e:
        print(f"执行错误: {e}")


--- 问题 1: 麻婆豆腐怎么做？回答少于100字 ---
回答: **正宗麻婆豆腐做法：**  
1. 嫩豆腐切块焯水备用。  
2. 锅中菜籽油烧热，下牛肉末炒酥，加郫县豆瓣酱、豆豉、辣椒面炒香。  
3. 加高汤、豆腐，小火慢烧入味。  
4. 勾薄芡，分两次收汁，撒花椒面、蒜苗。  
5. 出锅装盘，再撒一层花椒面即成。  
**关键：** 麻辣鲜香烫酥嫩，需用汉源花椒、郫县豆瓣。
{'question': '麻婆豆腐怎么做？回答少于100字'}

--- 问题 2: 白切鸡的正宗做法是什么？回答少于100字 ---
回答: 正宗白切鸡需选三黄鸡，清水浸熟而非煮沸。关键在“浸”：水沸后关火，放入整鸡浸20分钟，期间提吊两三次。熟后冰水急冻，皮脆肉嫩。斩件配姜葱蓉，骨髓带血丝为佳。
{'question': '白切鸡的正宗做法是什么？回答少于100字'}

--- 问题 3: 番茄炒蛋需要放糖吗？回答少于100字 ---
回答: 可以放，但非必需。放少量糖（约半茶匙）能中和番茄酸味、提升鲜味，使口感更柔和。传统做法中，上海等地习惯加糖，其他地区或家庭则可能只用盐调味。根据个人口味调整即可。
{'question': '番茄炒蛋需要放糖吗？回答少于100字'}
